# Model development

I will test 3 models (RandomForest, KNN and Linear Regression)

In [1]:
import pandas as pd
from taxipred.utils.constants import DATA_PATH

df = pd.read_csv(DATA_PATH/"cleaned_taxi_trip_pricing.csv")

print(f"{df.shape[0]} samples")
print(f"{df.shape[1]-1} features") # -1 because Trip_price is the label and not a feature

df.head(5)

683 samples
6 features


,Trip_Distance_km,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
0,19.35,Weekday,3.0,Low,Clear,53.82,36.2624
1,36.87,Weekend,1.0,High,Clear,37.27,52.9032
2,8.64,Weekend,2.0,Medium,Clear,89.33,60.2028
3,3.85,Weekday,4.0,High,Rain,5.05,11.2645
4,30.45,Weekday,3.0,High,Clear,110.33,NaN


Now i will split my trip_price so i have 2 seperate dataframes, one of the dataframe will have all of the data except where trip_price is NaN and the other dataframe will only have the NaN values of trip_price, so i later can predict the price using the model im gonna choose.

In [2]:
df_notna = df[df["Trip_Price"].notna()].copy()
df_notna

,Trip_Distance_km,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
0,19.35,Weekday,3.0,Low,Clear,53.82,36.2624
1,36.87,Weekend,1.0,High,Clear,37.27,52.9032
2,8.64,Weekend,2.0,Medium,Clear,89.33,60.2028
3,3.85,Weekday,4.0,High,Rain,5.05,11.2645
6,41.79,Weekend,3.0,High,Clear,86.95,88.1328
...,...,...,...,...,...,...,...
678,5.49,Weekend,4.0,Medium,Clear,58.39,34.4049
679,45.95,Weekday,4.0,Medium,Clear,61.96,62.1295
680,7.70,Weekday,3.0,Low,Rain,54.18,33.1236
681,47.56,Weekday,1.0,Low,Clear,114.94,61.2090


In [3]:
df_nan_values = df[df["Trip_Price"].isna()].copy()
df_nan_values.head(5)

,Trip_Distance_km,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
4,30.45,Weekday,3.0,High,Clear,110.33,NaN
5,48.53,Weekday,3.0,Low,Clear,79.94,NaN
17,4.19,Weekday,1.0,Low,Clear,69.06,NaN
61,38.78,Weekday,3.0,Medium,Clear,90.14,NaN
108,10.14,Weekday,4.0,Low,Rain,25.41,NaN


Now i will split the data (X is features and y is label), even the NaN value data

In [4]:
X, y = df_notna.drop("Trip_Price", axis="columns"), df_notna["Trip_Price"]
X.head(2), y.head(2)

(   Trip_Distance_km Day_of_Week  Passenger_Count Traffic_Conditions Weather  \
 0             19.35     Weekday              3.0                Low   Clear   
 1             36.87     Weekend              1.0               High   Clear   
 
    Trip_Duration_Minutes  
 0                  53.82  
 1                  37.27  ,
 0    36.2624
 1    52.9032
 Name: Trip_Price, dtype: float64)

In [5]:
X_nan_values = df_nan_values.drop(columns="Trip_Price").copy()
X_nan_values.head(5)

,Trip_Distance_km,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes
4,30.45,Weekday,3.0,High,Clear,110.33
5,48.53,Weekday,3.0,Low,Clear,79.94
17,4.19,Weekday,1.0,Low,Clear,69.06
61,38.78,Weekday,3.0,Medium,Clear,90.14
108,10.14,Weekday,4.0,Low,Rain,25.41


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((456, 6), (196, 6), (456,), (196,))

Now i need to encode the data so i have int_cols (columns that are type: int) and str_cols (columns that are object / type: str)

In [7]:
int_cols = X_train.select_dtypes(include="number").columns
str_cols = X_train.select_dtypes(exclude="number").columns

This code is Ai generated i just switched the variables to the ones i have, this code is converting kategorical data to numbers.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), int_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), str_cols),
    ]
)


Now I will start with the linear regression, this is with help from Ai. Using pipeline I dont need to do scaler.fit(X_train), scaler.transform(X_train) or scaler.transform(X_test), it's happening automaticly when using pipeline (as i understood it). Pipeline guarantees no data leakege and a cleaner code. I use Pipeline and ColumnTransformer because i have both numerical data and kategorical data. 

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

lr_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

In [10]:
lr_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformer

In [11]:
y_pred_lr = lr_model.predict(X_test)

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, y_pred_lr)
mse = mean_squared_error(y_test, y_pred_lr)
rmse = np.sqrt(mse)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}")

MAE: 11.76, MSE: 232.19, RMSE: 15.24


# KNN

I chatted with Ai to see how and where I should start in KNN and it told me to use the same X_train,X_test, y_train and y_test and that I should do a pipeline like I did before and finnish all my models before I compare them with eachother and test them on the test data

In [13]:
from sklearn.neighbors import KNeighborsRegressor

knn_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", KNeighborsRegressor(n_neighbors=5))
])

In [14]:
knn_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformer

In [15]:
y_pred_knn = knn_model.predict(X_test)

In [16]:
mae_knn = mean_absolute_error(y_test, y_pred_knn)
mse_knn = mean_squared_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))

print(f"MAE: {mae_knn:.2f}, MSE: {mse_knn:.2f}, RMSE: {rmse_knn:.2f}")

MAE: 13.06, MSE: 285.32, RMSE: 16.89


# RandomForest

In [17]:
from sklearn.ensemble import RandomForestRegressor

rf_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(
        n_estimators=200,
        random_state=42
    ))
])

In [18]:
rf_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformer

In [19]:
y_pred_rf = rf_model.predict(X_test)

In [20]:
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print(f"MAE: {mae_rf:.2f}, MSE: {mse_rf:.2f}, RMSE: {rmse_rf:.2f}")

MAE: 12.21, MSE: 253.36, RMSE: 15.92


Now I will compare the models to see wich suits the data the best. I want MAE, MSE and RMSE to be as low as possible, the model with the lowest results is the one that suits the data the best. 

In [21]:
results = pd.DataFrame({
    "Model": ["Linear Regression", "KNN", "Random Forest"],
    "MAE": [mae, mae_knn, mae_rf],
    "MSE": [mse, mse_knn, mse_rf], 
    "RMSE": [rmse, rmse_knn, rmse_rf]
})

results

,Model,MAE,MSE,RMSE
0,Linear Regression,11.758307,232.186031,15.237652
1,KNN,13.060933,285.321234,16.891454
2,Random Forest,12.205005,253.355923,15.917158


As we see Linear Regression has the lowest result in MAE, MSE and RMSE wich means that I will choose Linear Regression. 

In [22]:
lr_model.fit(X, y) 

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformer

Now I will predict the price of the NaN values I had in trip_price

In [23]:
predicted_prices = lr_model.predict(X_nan_values)
predicted_prices

array([72.51362322, 86.10925642, 27.63004368, 77.21592762, 25.6465067 ,
       75.77498694, 61.61382242, 64.66480318, 74.68474667, 67.73353858,
       91.47333172, 70.01936731, 63.76901824, 69.07148774, 39.20236753,
       26.14843047, 26.93629475, 51.67999023, 55.8643182 , 51.69982347,
       22.95781913, 43.82045851, 63.28487874, 27.7758598 , 42.52211233,
       46.4602083 , 33.75710923, 43.5325405 , 44.31489469, 74.95303017,
       48.16395967])

In [24]:
df_nan_values = df_nan_values.copy()
df_nan_values["Trip_Price_predicted"] = predicted_prices

df_nan_values.head()

,Trip_Distance_km,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price,Trip_Price_predicted
4,30.45,Weekday,3.0,High,Clear,110.33,NaN,72.513623
5,48.53,Weekday,3.0,Low,Clear,79.94,NaN,86.109256
17,4.19,Weekday,1.0,Low,Clear,69.06,NaN,27.630044
61,38.78,Weekday,3.0,Medium,Clear,90.14,NaN,77.215928
108,10.14,Weekday,4.0,Low,Rain,25.41,NaN,25.646507


In [25]:
df_nan_filled = df_nan_values.copy()
df_nan_filled["Trip_Price"] = df_nan_filled["Trip_Price_predicted"]
df_nan_filled = df_nan_filled.drop(columns=["Trip_Price_predicted"])

df_full = pd.concat([df_notna, df_nan_filled], axis=0).sort_index()
df_full.head()

,Trip_Distance_km,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Trip_Duration_Minutes,Trip_Price
0,19.35,Weekday,3.0,Low,Clear,53.82,36.262400
1,36.87,Weekend,1.0,High,Clear,37.27,52.903200
2,8.64,Weekend,2.0,Medium,Clear,89.33,60.202800
3,3.85,Weekday,4.0,High,Rain,5.05,11.264500
4,30.45,Weekday,3.0,High,Clear,110.33,72.513623


In [26]:
import joblib

joblib.dump(
    lr_model,
    DATA_PATH / "taxi_price_model.joblib",
    compress=("xz", 3),
    protocol=5
)

['C:\\Users\\norel\\Project\\taxi-prediction-fullstack-nore\\src\\taxipred\\data\\taxi_price_model.joblib']